In [1]:
from datetime import datetime, timedelta
import pandas as pd
import time
import psycopg2

In [14]:
starttime1 = time.time()
master_alarm_filename = r"C:\Users\com\Documents\facts\pms\doc\alarm_co-relation\Master_Alarm_file.xlsx"
master_alarm_df = pd.read_excel(master_alarm_filename, sheet_name = 'RAN')
print(master_alarm_df.shape)
print("Time Taken = ", time.time() - starttime1)

(94, 7)
Time Taken =  0.06801629066467285


In [ ]:
starttime1 = time.time()
hadoop_filename = r"C:\Users\com\Documents\facts\pms\doc\alarm_co-relation\Hadoop_Alarms_0106-2506-4.csv"
df1 = pd.read_csv(hadoop_filename)
print(df1.shape)
print("Time Taken = ", time.time() - starttime1)

In [15]:
starttime2 = time.time()
hadoop_filename = r"C:\Users\com\Documents\facts\pms\doc\alarm_co-relation\Hadoop_Alarms_0106-2506-4.csv"
column_use = ['originaleventtime', 'circle', 'domain_type', 'vendor', 'technology', 'nss_id', 'node_name', 'specific_problem_id', 'al_specific_problem', 'sub_specific_problem', 
              'additional_information', 'additionaltext', 'originalseverity', 'stateid']
hadoop_df = pd.read_csv(hadoop_filename, usecols=column_use)
print(hadoop_df.shape)
print("Time Taken = ", time.time() - starttime2)

(100002, 14)
Time Taken =  3.4179935455322266


C:\Users\com\AppData\Local\Temp\ipykernel_15304\4169823334.py:5: DtypeWarning: Columns (146) have mixed types. Specify dtype option on import or set low_memory=False.
  hadoop_df = pd.read_csv(hadoop_filename, usecols=column_use)


In [16]:
print(master_alarm_df.shape)
print(hadoop_df.shape)

# print(hadoop_df.head(3))
# hadoop_df.to_csv(r'C:\Users\com\Documents\facts\pms\doc\alarm_co-relation\final.csv', index=False)

(94, 7)
(100002, 14)


In [5]:

conn = psycopg2.connect(
    host="localhost",
    database="pms_test",
    user="postgres",
    password="admin"
)
# Create a cursor object
cur = conn.cursor()
print("cur = ", cur)


# Nokia
print("\n Before Filter")
print(hadoop_df.shape)
print(master_alarm_df.shape)

domain1 = "BSS"
vendor1 = "NOKIA"

hadoop_df = hadoop_df[hadoop_df['domain_type']==domain1]
hadoop_df = hadoop_df[hadoop_df['vendor']==vendor1]

master_alarm_df = master_alarm_df[master_alarm_df['Vendor']==vendor1]

print("\n After Filter")
print(hadoop_df.shape)
print(master_alarm_df.shape)


# for alarm_number in master_alarm_number:
for ma_index, ma_row in master_alarm_df.iterrows():
    ma_alarm_number = int(ma_row['Alarm number'])
    ma_conditon = str(ma_row['Conditon']).strip()
    ma_alarm_text = ma_row['Alarm text']
    ma_category = ma_row['Category']
    print("\nma_alarm_number = ", ma_alarm_number)
    print("ma_conditon = ", ma_conditon)

    # filter_hadoop_df = hadoop_df[hadoop_df['specific_problem_id']==ma_alarm_number]
    # print("\n",ma_alarm_number, filter_hadoop_df.shape)

    for hd_index, hd_row in hadoop_df.iterrows():
        hd_nssid = hd_row['nss_id']
        hd_specific_id = int(hd_row['specific_problem_id'])
        hd_datetime = hd_row['originaleventtime']
        hd_add_info_full = hd_row['additional_information']
        hd_add_info = hd_row['additional_information'].split("|")[0].strip()
        
        # if ma_alarm_number == hd_specific_id and ma_conditon == hd_add_info:
        if ma_alarm_number == hd_specific_id:
            # filter_hadoop_df = hadoop_df[hadoop_df['additional_information']==hd_add_info_full]
            filter_hadoop_df = hadoop_df[hadoop_df['additional_information']==hd_add_info_full]
            fhd_dates = filter_hadoop_df['originaleventtime'].tolist()

            if fhd_dates:
                fhd_dates = [datetime.strptime(date_str, '%m/%d/%Y %H:%M') for date_str in fhd_dates]
                min_date = min(fhd_dates)
                max_date = max(fhd_dates)
                date_difference = max_date - min_date
                if date_difference >= timedelta(days=15):
                    days_between = 'Yes'
                else:
                    days_between = 'No'
            else:
                days_between = 'No'

            insert_query = '''
            INSERT INTO alarm_corelation (nss_id, domain_type, vendor, category, alarm_number, supp_info, days_between)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            '''
            values = (
                hd_nssid, domain1, vendor1, ma_category, hd_specific_id, hd_add_info, days_between
            )
            # aa = cur.execute(insert_query, values)
            # print("Insert = ", aa)
            try:
                cur.execute(insert_query, values)
                conn.commit()
                # print("Insert successful")
            except Exception as e:
                conn.rollback()
                print(f"Error inserting data: {e}")


# Close cursor and connection
cur.close()
conn.close()
print("Endddddd")

cur =  <cursor object at 0x00000176627EB920; closed: 0>

 Before Filter
(100002, 14)
(94, 7)

 After Filter
(1922, 14)
(44, 7)

ma_alarm_number =  7115
ma_conditon =  No module fan detected
hd_nssid =  INDL004348
hd_nssid =  INDL004348
hd_nssid =  INDL004019
hd_nssid =  INDL004019
hd_nssid =  INDL004052
hd_nssid =  INDL000536
hd_nssid =  INDL000536
hd_nssid =  INDL006492
hd_nssid =  INDL006492
hd_nssid =  INDL001063
hd_nssid =  INDL006492
hd_nssid =  INDL006492
hd_nssid =  INDL001063
ok
Insert successful
hd_nssid =  IDDL100568
hd_nssid =  INDL000981
hd_nssid =  INDL002504
hd_nssid =  INDL002504
hd_nssid =  INDL002504
hd_nssid =  INDL401536
hd_nssid =  INDL000295
hd_nssid =  INDL000295
hd_nssid =  INDL402293
hd_nssid =  INDL007021
hd_nssid =  IDDL104940
ok
Insert successful
hd_nssid =  INDL001124
hd_nssid =  INDL000138
hd_nssid =  INDL402293
hd_nssid =  INDL405159
hd_nssid =  INDL405159
hd_nssid =  INDL402230
hd_nssid =  INDL000119
hd_nssid =  INDL001560
hd_nssid =  INDL001537
hd_nssid 

In [13]:

conn = psycopg2.connect(
    host="localhost",
    database="pms_test",
    user="postgres",
    password="admin"
)
# Create a cursor object
cur = conn.cursor()
print("cur = ", cur)


# Nokia
print("\n Before Filter")
print(hadoop_df.shape)
print(master_alarm_df.shape)

domain1 = "BSS"
vendor1 = "HUAWEI"

hadoop_df = hadoop_df[hadoop_df['domain_type']==domain1]
hadoop_df = hadoop_df[hadoop_df['vendor']==vendor1]

master_alarm_df = master_alarm_df[master_alarm_df['Vendor']==vendor1]

print("\n After Filter")
print(hadoop_df.shape)
print(master_alarm_df.shape)


# for alarm_number in master_alarm_number:
for ma_index, ma_row in master_alarm_df.iterrows():
    ma_alarm_number = int(ma_row['Alarm number'])
    ma_conditon = str(ma_row['Conditon']).strip()
    ma_alarm_text = ma_row['Alarm text']
    ma_category = ma_row['Category']
    print("\nma_alarm_number = ", ma_alarm_number)
    print("ma_conditon = ", ma_conditon)

    # filter_hadoop_df = hadoop_df[hadoop_df['specific_problem_id']==ma_alarm_number]
    # print("\n",ma_alarm_number, filter_hadoop_df.shape)

    for hd_index, hd_row in hadoop_df.iterrows():
        hd_nssid = hd_row['nss_id']
        hd_specific_id = int(hd_row['specific_problem_id'])
        hd_datetime = hd_row['originaleventtime']
        hd_add_info_full = hd_row['additional_information']
        hd_add_info = hd_row['additional_information'].split("|")[0].strip()
        
        # if ma_alarm_number == hd_specific_id and ma_conditon == hd_add_info:
        if ma_alarm_number == hd_specific_id:
            # filter_hadoop_df = hadoop_df[hadoop_df['additional_information']==hd_add_info_full]
            filter_hadoop_df = hadoop_df[hadoop_df['additional_information']==hd_add_info_full]
            fhd_dates = filter_hadoop_df['originaleventtime'].tolist()

            if fhd_dates:
                fhd_dates = [datetime.strptime(date_str, '%m/%d/%Y %H:%M') for date_str in fhd_dates]
                min_date = min(fhd_dates)
                max_date = max(fhd_dates)
                date_difference = max_date - min_date
                if date_difference >= timedelta(days=15):
                    days_between = 'Yes'
                else:
                    days_between = 'No'
            else:
                days_between = 'No'

            insert_query = '''
            INSERT INTO alarm_corelation (nss_id, domain_type, vendor, category, alarm_number, supp_info, days_between)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            '''
            values = (
                hd_nssid, domain1, vendor1, ma_category, hd_specific_id, hd_add_info, days_between
            )
            # aa = cur.execute(insert_query, values)
            # print("Insert = ", aa)
            try:
                cur.execute(insert_query, values)
                conn.commit()
                # print("Insert successful")
            except Exception as e:
                conn.rollback()
                print(f"Error inserting data: {e}")


# Close cursor and connection
cur.close()
conn.close()
print("Endddddd")

cur =  <cursor object at 0x00000176657252A0; closed: 0>

 Before Filter
(12902, 14)
(23, 7)

 After Filter
(12902, 14)
(23, 7)

ma_alarm_number =  26110
ma_conditon =  nan

ma_alarm_number =  25673
ma_conditon =  nan

ma_alarm_number =  25650
ma_conditon =  nan

ma_alarm_number =  26104
ma_conditon =  nan

ma_alarm_number =  26525
ma_conditon =  nan

ma_alarm_number =  25656
ma_conditon =  nan

ma_alarm_number =  25650
ma_conditon =  Node Temperature_Fan_Faulty

ma_alarm_number =  25652
ma_conditon =  Node Temperature_Fan_Faulty

ma_alarm_number =  25654
ma_conditon =  Node Temperature_Fan_Faulty

ma_alarm_number =  25655
ma_conditon =  Node Temperature_Fan_Faulty

ma_alarm_number =  25656
ma_conditon =  Node Temperature_Fan_Faulty

ma_alarm_number =  25713
ma_conditon =  Node Temperature_Fan_Faulty

ma_alarm_number =  26104
ma_conditon =  Node Temperature_Fan_Faulty

ma_alarm_number =  26525
ma_conditon =  Node Temperature_Fan_Faulty

ma_alarm_number =  26771
ma_conditon =  Node Tempe

In [17]:

conn = psycopg2.connect(
    host="localhost",
    database="pms_test",
    user="postgres",
    password="admin"
)
# Create a cursor object
cur = conn.cursor()
print("cur = ", cur)


# Nokia
print("\n Before Filter")
print(hadoop_df.shape)
print(master_alarm_df.shape)

domain1 = "BSS"
vendor1 = "ZTE"

hadoop_df = hadoop_df[hadoop_df['domain_type']==domain1]
hadoop_df = hadoop_df[hadoop_df['vendor']==vendor1]

master_alarm_df = master_alarm_df[master_alarm_df['Vendor']==vendor1]

print("\n After Filter")
print(hadoop_df.shape)
print(master_alarm_df.shape)


# for alarm_number in master_alarm_number:
for ma_index, ma_row in master_alarm_df.iterrows():
    ma_alarm_number = int(ma_row['Alarm number'])
    ma_conditon = str(ma_row['Conditon']).strip()
    ma_alarm_text = ma_row['Alarm text']
    ma_category = ma_row['Category']
    print("\nma_alarm_number = ", ma_alarm_number)
    print("ma_conditon = ", ma_conditon)

    # filter_hadoop_df = hadoop_df[hadoop_df['specific_problem_id']==ma_alarm_number]
    # print("\n",ma_alarm_number, filter_hadoop_df.shape)

    for hd_index, hd_row in hadoop_df.iterrows():
        hd_nssid = hd_row['nss_id']
        hd_specific_id = int(hd_row['specific_problem_id'])
        hd_datetime = hd_row['originaleventtime']
        hd_add_info_full = hd_row['additional_information']
        hd_add_info = hd_row['additional_information'].split("|")[0].strip()
        
        # if ma_alarm_number == hd_specific_id and ma_conditon == hd_add_info:
        if ma_alarm_number == hd_specific_id:
            # filter_hadoop_df = hadoop_df[hadoop_df['additional_information']==hd_add_info_full]
            filter_hadoop_df = hadoop_df[hadoop_df['additional_information']==hd_add_info_full]
            fhd_dates = filter_hadoop_df['originaleventtime'].tolist()

            if fhd_dates:
                fhd_dates = [datetime.strptime(date_str, '%m/%d/%Y %H:%M') for date_str in fhd_dates]
                min_date = min(fhd_dates)
                max_date = max(fhd_dates)
                date_difference = max_date - min_date
                if date_difference >= timedelta(days=15):
                    days_between = 'Yes'
                else:
                    days_between = 'No'
            else:
                days_between = 'No'

            insert_query = '''
            INSERT INTO alarm_corelation (nss_id, domain_type, vendor, category, alarm_number, supp_info, days_between)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            '''
            values = (
                hd_nssid, domain1, vendor1, ma_category, hd_specific_id, hd_add_info, days_between
            )
            # aa = cur.execute(insert_query, values)
            # print("Insert = ", aa)
            try:
                cur.execute(insert_query, values)
                conn.commit()
                # print("Insert successful")
            except Exception as e:
                conn.rollback()
                print(f"Error inserting data: {e}")


# Close cursor and connection
cur.close()
conn.close()
print("Endddddd")

cur =  <cursor object at 0x00000176628D4200; closed: 0>

 Before Filter
(100002, 14)
(94, 7)

 After Filter
(5, 14)
(14, 7)

ma_alarm_number =  198097062
ma_conditon =  Node Temperature_Fan_Faulty

ma_alarm_number =  198098466
ma_conditon =  Node Temperature_Fan_Faulty

ma_alarm_number =  198094409
ma_conditon =  Node Temperature_Fan_Faulty

ma_alarm_number =  198094406
ma_conditon =  Node Temperature_Fan_Faulty

ma_alarm_number =  198092071
ma_conditon =  Node Temperature_Fan_Faulty

ma_alarm_number =  198097062
ma_conditon =  Node Temperature_Fan_Faulty

ma_alarm_number =  198092557
ma_conditon =  Node Temperature_Fan_Faulty

ma_alarm_number =  198092069
ma_conditon =  FAN Module Faulty

ma_alarm_number =  198098111
ma_conditon =  FAN Module Faulty

ma_alarm_number =  198098474
ma_conditon =  FAN Module Faulty

ma_alarm_number =  198098114
ma_conditon =  FAN Module Faulty

ma_alarm_number =  198097063
ma_conditon =  FAN Module Faulty

ma_alarm_number =  198098476
ma_conditon =  FAN M